In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Đọc dữ liệu từ tệp CSV
df = pd.read_csv('tourist_attractions.csv')

# Hiển thị các cột của tập dữ liệu
print(df.columns)

# Tiền xử lý dữ liệu
# Chuyển đổi các giá trị văn bản thành dạng số nếu cần thiết, và xử lý các giá trị missing (nếu có)
df['Google review rating'] = pd.to_numeric(df['Google review rating'], errors='coerce')
df['Entrance Fee in INR'] = pd.to_numeric(df['Entrance Fee in INR'], errors='coerce')
df['Number of google review in lakhs'] = pd.to_numeric(df['Number of google review in lakhs'], errors='coerce')
#df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# Xử lý missing values
df = df.dropna()

# Lọc cộng tác sử dụng Surprise
# Chúng ta cần chuẩn bị dữ liệu theo định dạng user-item-rating
# Ở đây giả định mỗi người dùng đánh giá một địa điểm duy nhất

# Tạo ID giả định cho người dùng (vì dữ liệu không có ID người dùng)
df['user_id'] = df.index

# Chọn các cột cần thiết cho mô hình lọc cộng tác
ratings_dict = {
    'userID': df['user_id'],
    'itemID': df['Name'],
    'rating': df['Google review rating']
}
df_ratings = pd.DataFrame(ratings_dict)

# Định nghĩa đối tượng Reader và Dataset
reader = Reader(rating_scale=(df_ratings['rating'].min(), df_ratings['rating'].max()))
data = Dataset.load_from_df(df_ratings[['userID', 'itemID', 'rating']], reader)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
trainset, testset = train_test_split(data, test_size=0.2)

# Sử dụng mô hình SVD cho lọc cộng tác
algo = SVD()
algo.fit(trainset)

# Dự đoán và tính toán độ chính xác trên tập kiểm tra
predictions = algo.test(testset)
accuracy.rmse(predictions)

# Hàm để gợi ý địa điểm dựa trên lọc cộng tác
def collaborative_filtering_recommendations(user_id, num_recommendations=5):
    user_ratings = df_ratings[df_ratings['userID'] == user_id]
    user_unrated_items = df_ratings[~df_ratings['itemID'].isin(user_ratings['itemID'])]['itemID'].unique()
    
    predictions = [algo.predict(user_id, item_id) for item_id in user_unrated_items]
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:num_recommendations]
    
    return [pred.iid for pred in recommendations]

# Ví dụ về việc sử dụng hàm gợi ý
user_id = 10  # ID giả định của người dùng
recommendations = collaborative_filtering_recommendations(user_id)
print(f"Collaborative Filtering Recommendations for user {user_id}:")
for rec in recommendations:
    print(rec)

# Tạo hồ sơ người dùng dựa trên các thẻ
user_tags = {
    'beach': 5,
    'historical': 3,
    'adventure': 4
}

# Tạo hồ sơ điểm du lịch dựa trên các thẻ
item_profiles = df.set_index('Name')[['Zone', 'State', 'City', 'Type', 'Significance', 'Best Time to visit']]

# Mã hóa các hồ sơ này để phù hợp với hồ sơ người dùng
vectorizer = TfidfVectorizer()
item_profiles_encoded = vectorizer.fit_transform(item_profiles['Type'] + ' ' + item_profiles['Significance'] + ' ' + item_profiles['Best Time to visit'])

# Tính toán độ tương đồng giữa hồ sơ người dùng và các điểm du lịch
user_profile_encoded = vectorizer.transform([' '.join(user_tags.keys())])
cosine_similarities = cosine_similarity(user_profile_encoded, item_profiles_encoded).flatten()

# Lấy top các đề xuất
top_recommendations_indices = cosine_similarities.argsort()[-5:][::-1]
top_recommendations = item_profiles.iloc[top_recommendations_indices].index

print("Personalized recommendations based on tags:")
for rec in top_recommendations:
    print(rec)


Index(['Unnamed: 0', 'Zone', 'State', 'City', 'Name', 'Type',
       'Establishment Year', 'time needed to visit in hrs',
       'Google review rating', 'Entrance Fee in INR',
       'Airport with 50km Radius', 'Weekly Off', 'Significance',
       'DSLR Allowed', 'Number of google review in lakhs',
       'Best Time to visit'],
      dtype='object')
RMSE: 0.1693
Collaborative Filtering Recommendations for user 10:
Madhya Pradesh Tribal Museum
Charminar
Ellora Caves
Cellular Jail
Ujjayanta Palace
Personalized recommendations based on tags:
Ujjayanta Palace
Cooch Behar Palace
Laxmi Vilas Palace
Victoria Memorial
Bangalore Palace


In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Đọc dữ liệu từ tệp CSV
df = pd.read_csv('tourist_attractions.csv')

# Hiển thị các cột của tập dữ liệu
print(df.columns)

# Tiền xử lý dữ liệu
df['Google review rating'] = pd.to_numeric(df['Google review rating'], errors='coerce')
df['Entrance Fee in INR'] = pd.to_numeric(df['Entrance Fee in INR'], errors='coerce')
df['Number of google review in lakhs'] = pd.to_numeric(df['Number of google review in lakhs'], errors='coerce')
#df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
df = df.dropna()

# Tạo ID giả định cho người dùng (vì dữ liệu không có ID người dùng)
df['user_id'] = df.index

# Collaborative Filtering 
ratings_dict = {
    'userID': df['user_id'],
    'itemID': df['Name'],
    'rating': df['Google review rating']
}
df_ratings = pd.DataFrame(ratings_dict)

reader = Reader(rating_scale=(df_ratings['rating'].min(), df_ratings['rating'].max()))
data = Dataset.load_from_df(df_ratings[['userID', 'itemID', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)

algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

def collaborative_filtering_recommendations(user_id, num_recommendations=5):
    user_ratings = df_ratings[df_ratings['userID'] == user_id]
    user_unrated_items = df_ratings[~df_ratings['itemID'].isin(user_ratings['itemID'])]['itemID'].unique()
    
    predictions = [algo.predict(user_id, item_id) for item_id in user_unrated_items]
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:num_recommendations]
    
    return [(pred.iid, pred.est) for pred in recommendations]

# Tag-based Filtering
item_profiles = df.set_index('Name')[['Zone', 'State', 'City', 'Type', 'Significance', 'Best Time to visit']]
vectorizer = TfidfVectorizer()
item_profiles_encoded = vectorizer.fit_transform(item_profiles['Type'] + ' ' + item_profiles['Significance'] + ' ' + item_profiles['Best Time to visit'])

def tag_based_recommendations(user_tags, num_recommendations=5):
    user_profile_encoded = vectorizer.transform([' '.join(user_tags.keys())])
    cosine_similarities = cosine_similarity(user_profile_encoded, item_profiles_encoded).flatten()
    
    top_recommendations_indices = cosine_similarities.argsort()[-num_recommendations:][::-1]
    top_recommendations = item_profiles.iloc[top_recommendations_indices].index
    
    return [(item, cosine_similarities[idx]) for item, idx in zip(top_recommendations, top_recommendations_indices)]

# Hybrid Recommendation System
def hybrid_recommendations(user_id, user_tags, num_recommendations=5, alpha=0.5):
    cf_recommendations = collaborative_filtering_recommendations(user_id, num_recommendations * 2)
    tb_recommendations = tag_based_recommendations(user_tags, num_recommendations * 2)
    
    # Tạo từ điển đề xuất kết hợp
    recommendation_dict = {}
    
    for item, score in cf_recommendations:
        recommendation_dict[item] = alpha * score
    
    for item, score in tb_recommendations:
        if item in recommendation_dict:
            recommendation_dict[item] += (1 - alpha) * score
        else:
            recommendation_dict[item] = (1 - alpha) * score
    
    # Sắp xếp đề xuất dựa trên điểm tổng hợp
    sorted_recommendations = sorted(recommendation_dict.items(), key=lambda x: x[1], reverse=True)
    
    # Trả lại đề xuất N hàng đầu
    return sorted_recommendations[:num_recommendations]

# Ví dụ về việc sử dụng hệ thống đề xuất hybrid
user_id = 0  # ID giả định của người dùng
user_tags = {
    'beach': 5,
    'historical': 3,
    'adventure': 4
}

recommendations = hybrid_recommendations(user_id, user_tags)
print(f"Hybrid Recommendations for user {user_id}:")
for rec, score in recommendations:
    print(f"{rec}: {score}")


Index(['Unnamed: 0', 'Zone', 'State', 'City', 'Name', 'Type',
       'Establishment Year', 'time needed to visit in hrs',
       'Google review rating', 'Entrance Fee in INR',
       'Airport with 50km Radius', 'Weekly Off', 'Significance',
       'DSLR Allowed', 'Number of google review in lakhs',
       'Best Time to visit'],
      dtype='object')
RMSE: 0.1715
Hybrid Recommendations for user 0:
Ujjayanta Palace: 2.41111097931328
Hazarduari Palace: 2.3914599331826
Ellora Caves: 2.3903268238181625
Ajanta Caves: 2.3882501000719873
Buddh International Circuit: 2.1853467114845566
